# 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.getcwd()), ".."))
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from certainty import EventSentence, EventType, load_events, id2label, label2id

2025-04-05 15:48:11.914207: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-05 15:48:12.160845: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-05 15:48:13.218279: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
train_events, dev_events, test_events = load_events('en_train.json', 'en_dev.json', 'en_test.json')

In [5]:
train_df = pd.DataFrame(train_events)
dev_df = pd.DataFrame(dev_events)

In [6]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report

In [7]:
y_train = train_df['label'].to_numpy()
y_dev = dev_df['label'].to_numpy()

In [30]:
model = DummyClassifier(strategy="most_frequent")

In [31]:
model.fit(None, y_train)

DummyClassifier(strategy='most_frequent')

In [32]:
y_pred = model.predict(y_dev)

In [39]:
len(y_pred)

496

In [41]:
import numpy as np

In [43]:
y_pred_op = np.zeros(496)

In [12]:
report = classification_report(y_dev, y_pred, zero_division=0.0, output_dict=True)

In [44]:
report2 = classification_report(y_dev, y_pred_op, zero_division=0.0, output_dict=True)

In [45]:
report

{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 140},
 '1': {'precision': 0.717741935483871,
  'recall': 1.0,
  'f1-score': 0.835680751173709,
  'support': 356},
 'accuracy': 0.717741935483871,
 'macro avg': {'precision': 0.3588709677419355,
  'recall': 0.5,
  'f1-score': 0.4178403755868545,
  'support': 496},
 'weighted avg': {'precision': 0.5151534859521332,
  'recall': 0.717741935483871,
  'f1-score': 0.599803119794033,
  'support': 496}}

In [46]:
report2

{'0': {'precision': 0.28225806451612906,
  'recall': 1.0,
  'f1-score': 0.4402515723270441,
  'support': 140},
 '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 356},
 'accuracy': 0.28225806451612906,
 'macro avg': {'precision': 0.14112903225806453,
  'recall': 0.5,
  'f1-score': 0.22012578616352205,
  'support': 496},
 'weighted avg': {'precision': 0.07966961498439128,
  'recall': 0.28225806451612906,
  'f1-score': 0.12426455670521407,
  'support': 496}}

In [69]:
total = 356+140

In [73]:
(356/total)+(140/total)

1.0

In [82]:
print((pd.DataFrame({"Factuality": ['Asserted', 'Macro avg', 'Weighted avg', 'Other', 'Macro avg', 'Weighted avg'],
             "P": [report['1']['precision'], 
                   report['macro avg']['precision'],
                   report['weighted avg']['precision'],
                   report2['0']['precision'], 
                   report2['macro avg']['precision'],
                   report2['weighted avg']['precision']],
              "R": [report['1']['recall'], 
                   report['macro avg']['recall'],
                   report['weighted avg']['recall'],
                   report2['0']['recall'], 
                   report2['macro avg']['recall'],
                   report2['weighted avg']['recall']],
              "F1": [report['1']['f1-score'], 
                     report['macro avg']['f1-score'],
                     report['weighted avg']['f1-score'],
                     report2['0']['f1-score'], 
                     report2['macro avg']['f1-score'],
                     report2['weighted avg']['f1-score']]
                    }).set_index('Factuality').mul(100)).round(1).to_latex())

\begin{tabular}{lrrr}
\toprule
{} &     P &      R &    F1 \\
Factuality   &       &        &       \\
\midrule
Asserted     &  71.8 &  100.0 &  83.6 \\
Macro avg    &  35.9 &   50.0 &  41.8 \\
Weighted avg &  51.5 &   71.8 &  60.0 \\
Other        &  28.2 &  100.0 &  44.0 \\
Macro avg    &  14.1 &   50.0 &  22.0 \\
Weighted avg &   8.0 &   28.2 &  12.4 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_21305/4225616045.py:20: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  }).set_index('Factuality').mul(100)).round(1).to_latex())


In [24]:
z['Macro avg'] = x
z['Weighted avg'] = y

In [26]:
z.T

,precision,recall,f1-score,support
0,0.000000,0.000000,0.000000,140.0
1,0.717742,1.000000,0.835681,356.0
Macro avg,0.358871,0.500000,0.417840,496.0
Weighted avg,0.515153,0.717742,0.599803,496.0


In [18]:
y.loc['Accuracy'] = ["", "", report['accuracy']]

In [19]:
y = y.rename({'f1-score': 'f1', 'precision': "Precision", "recall": "Recall"}, axis=0)

In [20]:
print(y.to_latex(bold_rows=True))

\begin{tabular}{lllr}
\toprule
{} &      0 &         1 &   macro\_avg \\
\midrule
\textbf{Precision} &    0.0 &  0.717742 &    0.358871 \\
\textbf{Recall   } &    0.0 &       1.0 &    0.500000 \\
\textbf{f1       } &    0.0 &  0.835681 &    0.417840 \\
\textbf{support  } &  140.0 &     356.0 &  496.000000 \\
\textbf{Accuracy } &        &           &    0.717742 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_21305/3862831697.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(y.to_latex(bold_rows=True))


In [22]:
y[['macro_avg']].to_csv('../results/metrics_majority_baseline.csv')

In [29]:
df = pd.DataFrame({"Majority class": [y.loc['Precision'][1], y.loc['Precision'][0], y.loc['Recall'][1], y.loc['Recall'][0], y.loc['f1'][1], y.loc['f1'][0], y.loc['Accuracy']['macro_avg']]}, 
                  ["Precision Asserted", "Precision Other", "Recall Asserted", "Recall Other", "f1 Asserted", "f1 Other", "Accuracy"])

In [31]:
df.to_csv('../results/majority_baseline.csv')